# EEG alignment

In [1]:
from __future__ import division
import csv
import numpy as np
import matplotlib.pylab as plt
from scipy.signal import resample
import mne
import pandas as pd
import seaborn as sns
from scipy.signal import hilbert
#from utils import circular_histogram
#import pycircstat
import os
from datetime import datetime

In [2]:
%run utlis.ipynb

In [3]:
def data_preprocessing(patient_id,channel):
    patient_data=str(patient_id);

    EEG_dir='D:/IM 680/byteflies_dots/'+patient_data+'/EEG File'
    EEG_dir=EEG_dir.replace('/', '\\')
    print(EEG_dir)


    EEG1E_data = pd.read_csv( os.path.join(EEG_dir, 'BF'+patient_data+'_1'+channel+'_EEG1_im680.csv')) 
    EEG1E2_data = pd.read_csv( os.path.join(EEG_dir,'BF'+patient_data+'_2'+channel+'_EEG1_im680.csv')) 
    EEG2E_data = pd.read_csv( os.path.join(EEG_dir, 'BF'+patient_data+'_1'+channel+'_EEG2_im680.csv')) 
    EEG2E2_data = pd.read_csv( os.path.join(EEG_dir,'BF'+patient_data+'_2'+channel+'_EEG2_im680.csv')) 

    IMU_dir='D:/IM 680/byteflies_dots/'+patient_data+'/IMU File'
    IMU_dir=IMU_dir.replace('/', '\\')
    ACC_data = pd.read_csv( os.path.join(IMU_dir, 'BF'+patient_data+'_1'+channel+'_ACC_im680.csv')) 
    ACC2_data = pd.read_csv( os.path.join(IMU_dir,'BF'+patient_data+'_2'+channel+'_ACC_im680.csv')) 
    GRY_data = pd.read_csv( os.path.join(IMU_dir, 'BF'+patient_data+'_1'+channel+'_GYR_im680.csv')) 
    GRY2_data = pd.read_csv( os.path.join(IMU_dir,'BF'+patient_data+'_1'+channel+'_GYR_im680.csv'))
    #find common time for all EEG GRY and ACC data
    start_time=max(ACC_data['time'][0],GRY_data['time'][0],
               ACC2_data['time'][0],GRY2_data['time'][0],
               EEG1E_data['time'][0],EEG1E2_data['time'][0],
               EEG2E_data['time'][0],EEG2E2_data['time'][0])

    end_time = min( ACC_data['time'].values[-1],  GRY_data['time'].values[-1],
                ACC2_data['time'].values[-1], GRY2_data['time'].values[-1],
                EEG1E_data['time'].values[-1],EEG1E2_data['time'].values[-1],
                EEG2E_data['time'].values[-1],EEG2E2_data['time'].values[-1])

    step=0.01;sfreq=int(1/step);
    
    time=np.arange(start=start_time,stop=end_time,step=step)
    #apply conversion factor for all the data
    #resample the data inbetween the start_time and end_time with step
    #filter the resampled data with sampling frequency 1/step and h_freq=0.1 and l_freq=40

    
    GRY_factor=0.0305;EEG_factor=0.4808;ACC_factor=0.0002;

    GRY_1EX = mne.filter.filter_data(data= resample(x=GRY_factor*GRY_data['channel1'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_1EY = mne.filter.filter_data(data= resample(x=GRY_factor*GRY_data['channel2'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_1EZ = mne.filter.filter_data(data= resample(x=GRY_factor*GRY_data['channel3'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_2EX = mne.filter.filter_data(data= resample(x=GRY_factor*GRY2_data['channel1'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_2EY = mne.filter.filter_data(data= resample(x=GRY_factor*GRY2_data['channel2'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_2EZ = mne.filter.filter_data(data= resample(x=GRY_factor*GRY2_data['channel3'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)

    ACC_1EX = mne.filter.filter_data(data= resample(x=ACC_factor*ACC_data['channel1'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_1EY = mne.filter.filter_data(data= resample(x=ACC_factor*ACC_data['channel2'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_1EZ = mne.filter.filter_data(data= resample(x=ACC_factor*ACC_data['channel3'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_2EX = mne.filter.filter_data(data= resample(x=ACC_factor*ACC2_data['channel1'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_2EY = mne.filter.filter_data(data= resample(x=ACC_factor*ACC2_data['channel2'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_2EZ = mne.filter.filter_data(data= resample(x=ACC_factor*ACC2_data['channel3'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)

    EEG_1E1 = mne.filter.filter_data(data= resample(x=EEG_factor*EEG1E_data['channel'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    EEG_1E2 = mne.filter.filter_data(data= resample(x=EEG_factor*EEG1E2_data['channel'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    EEG_2E1 = mne.filter.filter_data(data= resample(x=EEG_factor*EEG2E_data['channel'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    EEG_2E2 = mne.filter.filter_data(data= resample(x=EEG_factor*EEG2E2_data['channel'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)

    d = {'time':time, 'GRY 1EX':GRY_1EX, 'GRY 1EY':GRY_1EY,'GRY 1EZ':GRY_1EZ,
                  'GRY 2EX':GRY_2EX, 'GRY 2EY':GRY_2EY,'GRY 2EZ':GRY_2EZ,
                  'ACC 1EX':ACC_1EX, 'ACC 1EY':ACC_1EY,'ACC 1EZ':ACC_1EZ,
                  'ACC 2EX':ACC_2EX, 'ACC 2EY':ACC_2EY,'ACC 2EZ':ACC_2EZ,
    'EEG 1E1':EEG_1E1 ,'EEG 1E2':EEG_1E2,'EEG 2E1': EEG_2E1,'EEG 2E2': EEG_2E2
    }
    df = pd.DataFrame(data=d)

    #read the starting time point for PSG and BFDS data
    path='D:\IM 680\supporting_documents'
    PSG_SleepStage_Summary=pd.read_excel(path+'\IM680_PSG_SleepStage_Summary_FINAL.xlsx')
    PSG_StartTimestamps=pd.read_excel(path+'\IM680_BFSD_PSG_StartTimestamps_DateShifted_FINAL.xlsx')
    PSG_SleepStage_Data=pd.read_excel(path+'\IM680_PSG_SleepStage_Data_FINAL.xlsx')

    Data_pd=PSG_SleepStage_Data[PSG_SleepStage_Data['SUBID']==patient_id]
    print(len(Data_pd))
    if len(Data_pd)==0:
        print('There is no sleep stage data for this patient: '+patient_data)
        return df
    StartTime_1351=PSG_StartTimestamps[PSG_StartTimestamps['SUBID']==patient_id].values[0][1]

    StartTime_1351_PSG=PSG_StartTimestamps[PSG_StartTimestamps['SUBID']==patient_id].values[0][3]
    #calculate the difference between PSG adn BFDS data, we know
    time_format = '%Y-%m-%d %H:%M:%S.%f'
    BFSD_time = datetime.utcfromtimestamp(int(StartTime_1351)).strftime(time_format)
    PSG_time = datetime.utcfromtimestamp(int(StartTime_1351_PSG)).strftime(time_format)
    print(BFSD_time+' : This the time on Start of BFSD file.' )
    print(PSG_time +' : This the time on the PSG.')

    TimeDifference=(datetime.utcfromtimestamp(int(StartTime_1351_PSG))-datetime.utcfromtimestamp(int(StartTime_1351))).seconds

    start=df['time'][0]
    df['time']=df['time'].apply(lambda x: x-start)

    print('we need to throw out the first '+ str( TimeDifference)+'s in the BFDS file')

    df=df.iloc[int(TimeDifference/step):,:].reset_index()
    
    start=np.min(df['time'])
    df['time']=df['time'].apply(lambda x: x-start)

    df.rename(columns={'index':'sleep stage'},inplace=True)
    
    stage_list=np.repeat(Data_pd['Stage'].values, 30*100)

    stage_code_list=np.repeat(Data_pd['Stage_Coded'].values, 30*100)
    
    df['sleep stage']=np.nan
    #we also need to drop the later time when we don't have PSG data anymore
    df['sleep stage'].iloc[:len(stage_list)]=stage_list

    df.dropna(inplace=True)

    df['Sleep stage code']=stage_code_list
    return df
    #df.to_csv('BFDS'+patient_data+'.csv')

In [4]:
patient_id=1351;channel='E';

In [5]:
df1351=data_preprocessing(patient_id,channel)

D:\IM 680\byteflies_dots\1351\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

985
2021-08-20 22:24:53.000000 : This the time on Start of BFSD file.
2021-08-21 01:22:40.000000 : This the time on the PSG.
we need to throw out the first 10667s in the BFDS file


In [28]:
df1351[]

,sleep stage,time,GRY 1EX,GRY 1EY,GRY 1EZ,GRY 2EX,GRY 2EY,GRY 2EZ,ACC 1EX,ACC 1EY,ACC 1EZ,ACC 2EX,ACC 2EY,ACC 2EZ,EEG 1E1,EEG 1E2,EEG 2E1,EEG 2E2,Sleep stage code
0,L,0.00,-0.231904,-0.287483,0.073691,-0.231904,-0.287483,0.073691,-0.288648,-0.747749,-0.069214,-0.338225,-0.723022,-0.147072,45743.588203,34412.852071,-30330.877315,-31851.562020,99
1,L,0.01,-0.236062,-0.269708,0.096751,-0.236062,-0.269708,0.096751,-0.289225,-0.748662,-0.070143,-0.338510,-0.723412,-0.145994,34828.481951,41003.959484,-19782.481662,-35290.820829,99
2,L,0.02,-0.242968,-0.255912,0.069765,-0.242968,-0.255912,0.069765,-0.289382,-0.747597,-0.070042,-0.338932,-0.723129,-0.147482,43104.120627,35455.469369,-31094.952163,-32166.148378,99
3,L,0.03,-0.228555,-0.307415,0.077582,-0.228555,-0.307415,0.077582,-0.288451,-0.748481,-0.069502,-0.338093,-0.723295,-0.146516,38723.235184,36025.949655,-21904.397905,-32566.105828,99
4,L,0.04,-0.249284,-0.229847,0.105791,-0.249284,-0.229847,0.105791,-0.289717,-0.748158,-0.070361,-0.338888,-0.723195,-0.146324,40697.803612,41643.920424,-27574.943164,-34890.004375,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954995,L,29549.95,0.154728,-0.714566,-0.198503,0.154728,-0.714566,-0.198503,-0.128785,0.723991,-0.406115,-0.034111,0.669180,-0.471776,-29911.608175,66645.683678,-225896.677316,-115415.745513,99
2954996,L,29549.96,0.185186,-0.668487,-0.172196,0.185186,-0.668487,-0.172196,-0.128465,0.724799,-0.406191,-0.033742,0.669207,-0.471952,-32471.786594,59950.609282,-235421.096457,-109397.649319,99
2954997,L,29549.97,0.155481,-0.712882,-0.203866,0.155481,-0.712882,-0.203866,-0.129231,0.723195,-0.405242,-0.034987,0.668982,-0.472950,-33769.307904,64203.936503,-232466.135203,-113975.540005,99
2954998,L,29549.98,0.194069,-0.682276,-0.192598,0.194069,-0.682276,-0.192598,-0.128684,0.725003,-0.406810,-0.033701,0.668918,-0.471219,-32582.466549,58942.432121,-228590.982933,-111324.255077,99


In [6]:
df1351.to_csv('D:/IM 680/byteflies_dots/'+str(patient_id)+'.csv')

In [7]:
df2510=data_preprocessing(2510,'D')

D:\IM 680\byteflies_dots\2510\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

925
2021-01-25 20:21:01.000000 : This the time on Start of BFSD file.
2021-01-26 02:20:47.000000 : This the time on the PSG.
we need to throw out the first 21586s in the BFDS file


In [8]:
df2510.to_csv('D:/IM 680/byteflies_dots/'+str(2510)+'.csv')

In [9]:
df2939=data_preprocessing(2939,'E')

D:\IM 680\byteflies_dots\2939\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

889
2021-04-20 22:02:51.000000 : This the time on Start of BFSD file.
2021-04-21 02:22:16.000000 : This the time on the PSG.
we need to throw out the first 15565s in the BFDS file


In [10]:
df2939.to_csv('D:/IM 680/byteflies_dots/'+str(2939)+'.csv')

In [11]:
df4301=data_preprocessing(4301,'D')

D:\IM 680\byteflies_dots\4301\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

925
2020-11-13 20:09:13.000000 : This the time on Start of BFSD file.
2020-11-14 01:49:11.000000 : This the time on the PSG.
we need to throw out the first 20398s in the BFDS file


In [12]:
df4301.to_csv('D:/IM 680/byteflies_dots/'+str(4301)+'.csv')

In [13]:
df5852=data_preprocessing(5852,'C')

D:\IM 680\byteflies_dots\5852\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

864
2020-12-01 22:45:55.000000 : This the time on Start of BFSD file.
2020-12-02 02:27:23.000000 : This the time on the PSG.
we need to throw out the first 13288s in the BFDS file


In [14]:
df5852.to_csv('D:/IM 680/byteflies_dots/'+str(5852)+'.csv')

In [15]:
df5995=data_preprocessing(5995,'D')

D:\IM 680\byteflies_dots\5995\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

1059
2021-09-24 22:29:34.000000 : This the time on Start of BFSD file.
2021-09-25 01:18:11.000000 : This the time on the PSG.
we need to throw out the first 10117s in the BFDS file


In [16]:
df7257=data_preprocessing(7257,'D')

D:\IM 680\byteflies_dots\7257\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

943
2021-05-07 20:52:47.000000 : This the time on Start of BFSD file.
2021-05-08 02:12:19.000000 : This the time on the PSG.
we need to throw out the first 19172s in the BFDS file


In [17]:
df7257.to_csv('D:/IM 680/byteflies_dots/'+str(7257)+'.csv')

note that for patient 7838, I change the file names to be channel A so it is easier for me to read the files in

In [18]:
df7838=data_preprocessing(7838,'A')

D:\IM 680\byteflies_dots\7838\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

904
2021-05-28 23:29:11.000000 : This the time on Start of BFSD file.
2021-05-29 01:29:44.000000 : This the time on the PSG.
we need to throw out the first 7233s in the BFDS file


In [19]:
df7838.to_csv('D:/IM 680/byteflies_dots/'+str(7838)+'.csv')

In [20]:
df7844=data_preprocessing(7844,'E')

D:\IM 680\byteflies_dots\7844\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

890
2021-07-16 20:58:57.000000 : This the time on Start of BFSD file.
2021-07-17 02:06:49.000000 : This the time on the PSG.
we need to throw out the first 18472s in the BFDS file


In [21]:
df7844.to_csv('D:/IM 680/byteflies_dots/'+str(7844)+'.csv')

In [22]:
patient_id=5995;channel='D';

In [23]:
df5995=data_preprocessing(patient_id,channel)

D:\IM 680\byteflies_dots\5995\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

1059
2021-09-24 22:29:34.000000 : This the time on Start of BFSD file.
2021-09-25 01:18:11.000000 : This the time on the PSG.
we need to throw out the first 10117s in the BFDS file


In [24]:
df5995.to_csv('D:/IM 680/byteflies_dots/'+str(5995)+'.csv')

In [25]:
df8898=data_preprocessing(8898,'D')

D:\IM 680\byteflies_dots\8898\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

873
2020-07-24 22:34:01.000000 : This the time on Start of BFSD file.
2020-07-25 02:45:42.000000 : This the time on the PSG.
we need to throw out the first 15101s in the BFDS file


In [26]:
df8898.to_csv('D:/IM 680/byteflies_dots/'+str(8898)+'.csv')

In [27]:
df8898

,sleep stage,time,GRY 1EX,GRY 1EY,GRY 1EZ,GRY 2EX,GRY 2EY,GRY 2EZ,ACC 1EX,ACC 1EY,ACC 1EZ,ACC 2EX,ACC 2EY,ACC 2EZ,EEG 1E1,EEG 1E2,EEG 2E1,EEG 2E2,Sleep stage code
0,L,0.00,-0.999844,-0.587421,-0.269927,-0.999844,-0.587421,-0.269927,-0.807135,0.186466,0.152020,-0.620254,0.142496,0.427188,-186984.641183,-128357.101749,-79151.288538,62719.785683,99
1,L,0.01,-0.933865,-0.576127,-0.318000,-0.933865,-0.576127,-0.318000,-0.807008,0.186078,0.153840,-0.620003,0.142102,0.426689,-193650.197745,-124430.878150,-76689.301663,60402.209198,99
2,L,0.02,-0.973136,-0.594017,-0.281253,-0.973136,-0.594017,-0.281253,-0.805755,0.185941,0.152414,-0.620283,0.142578,0.426843,-189343.046006,-123651.324816,-76367.902161,59660.955435,99
3,L,0.03,-0.968001,-0.565442,-0.287911,-0.968001,-0.565442,-0.287911,-0.808069,0.186712,0.153300,-0.620192,0.141818,0.426952,-191583.614100,-131573.758138,-78106.813322,63123.048011,99
4,L,0.04,-0.948906,-0.598289,-0.309204,-0.948906,-0.598289,-0.309204,-0.805392,0.185472,0.152815,-0.620141,0.142804,0.426706,-190171.345848,-117091.811403,-76810.470898,58130.620365,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618995,L,26189.95,-1.098962,-0.602494,-0.236597,-1.098962,-0.602494,-0.236597,-0.653940,-0.032603,0.510382,-0.466415,0.061186,0.600472,-169854.774953,92962.780867,-185088.619462,-1451.582917,99
2618996,L,26189.96,-1.212067,-0.589044,-0.245950,-1.212067,-0.589044,-0.245950,-0.653853,-0.032560,0.509246,-0.466604,0.061110,0.600446,-179871.562500,90736.272625,-188616.131072,-8324.060534,99
2618997,L,26189.97,-1.144401,-0.627104,-0.234441,-1.144401,-0.627104,-0.234441,-0.653442,-0.032730,0.510000,-0.466231,0.061597,0.600163,-173502.662522,95470.558373,-189289.713291,-2538.335654,99
2618998,L,26189.98,-1.146676,-0.589289,-0.252129,-1.146676,-0.589289,-0.252129,-0.654113,-0.032343,0.509849,-0.466646,0.060711,0.600208,-176442.703450,90097.446657,-185323.860248,-5441.425105,99
